In [1]:
import os
import numpy as np
import pandas as pd
import torch
import shutil
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import json
import glob
import random
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision import models
from torchvision.models import ResNet50_Weights
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
base_dir = 'C:/Users/samue/it_plantas'
train_dir = os.path.join(base_dir, "images_filtered_train")
val_dir = os.path.join(base_dir, "images_filtered_val")
test_dir = os.path.join(base_dir, 'images_filtered_test')

In [3]:
num_train_images = len(glob.glob(train_dir + '/*/*.jpg'))
num_val_images = len(glob.glob(val_dir + '/*/*.jpg'))
num_test_images = len(glob.glob(test_dir + '/*/*.jpg'))

print('Total training images:', num_train_images)
print('Total validation images:', num_val_images)
print('Total test images:', num_test_images)
print('Total de imagens:', num_train_images+num_val_images+num_test_images)

Total training images: 192642
Total validation images: 24174
Total test images: 24169
Total de imagens: 240985


In [4]:
def listar_subdiretorios(dir):
    return [os.path.join(dir, nome) 
        for nome in os.listdir(dir) 
            if os.path.isdir(os.path.join(dir, nome))]

# Listar subdiretórios em train, val e test
train_subdirs = listar_subdiretorios(train_dir)
val_subdirs = listar_subdiretorios(val_dir)
test_subdirs = listar_subdiretorios(test_dir)

In [5]:
with open('C:/Users/samue/codigos/IT_PLANTAS/new_names.json') as f:
    data_json = json.load(f)

In [ ]:
def name_dir_count_images(directories, plant_dict):
    new_paths = []
    counts = {}  # Dicionário para armazenar o nome da planta e a contagem de imagens
    total_images = 0  # Variável para armazenar o total de imagens

    for dir_path in directories:
        plant_name = os.path.basename(dir_path)  # Extrai o nome da planta diretamente do caminho do diretório
        
        if plant_name in plant_dict.values():  # Verifica se o nome está no dicionário de plantas
            new_paths.append(dir_path)  # Adiciona o caminho original à lista de novos caminhos

            # Contar imagens no diretório
            image_count = len([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])
            counts[plant_name] = image_count  # Adiciona a contagem ao nome da planta
            total_images += image_count

        else:
            print(f"Planta: {plant_name} não encontrada no dicionário.")

    return new_paths, counts, total_images

# Exemplo de uso
new_train_subdirs, image_counts, total_images = name_dir_count_images(test_subdirs, data_json)

# Exibir a quantidade de imagens para cada planta
for plant_name, count in image_counts.items():
    print(f"Planta: {plant_name}, Quantidade de imagens: {count}")
print(f"Total de imagens: {total_images}")

In [ ]:
new_train_subdirs, image_counts, total_images = name_dir_count_images(val_subdirs, data_json)
for plant_name, count in image_counts.items():
    print(f"Planta: {plant_name}, Quantidade de imagens: {count}")
print(f"Total de imagens: {total_images}")

In [ ]:
new_train_subdirs, image_counts, total_images = name_dir_count_images(train_subdirs, data_json)

for plant_name, count in image_counts.items():
    print(f"Planta: {plant_name}, Quantidade de imagens: {count}")
print(f"Total de imagens: {total_images}")

In [9]:
def cont_image(directory):
    return sum([len(files) for _, _, files in os.walk(directory)])

In [10]:
train_cont = cont_image(train_dir)
val_cont = cont_image(val_dir)
test_cont = cont_image(test_dir)

total_images = train_cont + val_cont + test_cont

print(f"Treino: {train_cont} image ({(train_cont / total_images) * 100:.2f}%)")
print(f"Validação: {val_cont} image ({(val_cont / total_images) * 100:.2f}%)")
print(f"Teste: {test_cont} image ({(test_cont / total_images) * 100:.2f}%)")

Treino: 192642 image (79.94%)
Validação: 24174 image (10.03%)
Teste: 24169 image (10.03%)


In [11]:
# Definindo as transformações de dados
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Carregar os dados para treino e validação
image_datasets = {
    'train': datasets.ImageFolder(root=train_dir, transform=data_transforms['train']),
    'val': datasets.ImageFolder(root=val_dir, transform=data_transforms['val'])
}

# DataLoaders para carregar os dados em lotes
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
    for x in ['train', 'val']
}

In [ ]:
# Verificar as classes e definir o número de classes
class_names = image_datasets['train'].classes
num_classes = len(class_names)  # Obter a quantidade de classes
print(f"Classes encontradas: {class_names}")
print(f"Quantidade de classes: {num_classes}")

In [13]:
# Carregar o modelo ResNet50 com os pesos mais recentes do ImageNet
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

# Congelar as camadas convolucionais
for param in model.parameters():
    param.requires_grad = False

# Substituir a última camada para o número correto de classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Enviar o modelo para a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [14]:
# Definir a função de perda
criterion = nn.CrossEntropyLoss()

# Definir o otimizador (apenas para os parâmetros da última camada)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [15]:
#verificanso a GPU
print(torch.cuda.is_available())

True


In [16]:
# Função para treinar o modelo
def train_model(model, criterion, optimizer, dataloaders, device, num_epochs=20):
    model.train()  # Colocar o modelo em modo de treino
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zerar os gradientes
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass e otimização
            loss.backward()
            optimizer.step()

            # Estatísticas
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders['train'].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders['train'].dataset)

        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

# Treinar o modelo
train_model(model, criterion, optimizer, dataloaders, device, num_epochs=20)


Epoch 0/19, Loss: 1.7677, Accuracy: 0.5710
Epoch 1/19, Loss: 1.4318, Accuracy: 0.6422
Epoch 2/19, Loss: 1.3454, Accuracy: 0.6626
Epoch 3/19, Loss: 1.2883, Accuracy: 0.6740
Epoch 4/19, Loss: 1.2380, Accuracy: 0.6860
Epoch 5/19, Loss: 1.2039, Accuracy: 0.6947
Epoch 6/19, Loss: 1.1700, Accuracy: 0.7014
Epoch 7/19, Loss: 1.1407, Accuracy: 0.7074
Epoch 8/19, Loss: 1.1193, Accuracy: 0.7114
Epoch 9/19, Loss: 1.0953, Accuracy: 0.7151
Epoch 10/19, Loss: 1.0760, Accuracy: 0.7216
Epoch 11/19, Loss: 1.0508, Accuracy: 0.7264
Epoch 12/19, Loss: 1.0431, Accuracy: 0.7271
Epoch 13/19, Loss: 1.0239, Accuracy: 0.7324
Epoch 14/19, Loss: 1.0136, Accuracy: 0.7332
Epoch 15/19, Loss: 1.0002, Accuracy: 0.7372
Epoch 16/19, Loss: 0.9869, Accuracy: 0.7402
Epoch 17/19, Loss: 0.9755, Accuracy: 0.7415
Epoch 18/19, Loss: 0.9633, Accuracy: 0.7449
Epoch 19/19, Loss: 0.9577, Accuracy: 0.7463


In [17]:
def evaluate_model(model, dataloaders, device):
    model.eval()  # Colocar o modelo em modo de avaliação
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

    accuracy = running_corrects.double() / len(dataloaders['val'].dataset)
    print(f'Validation Accuracy: {accuracy:.4f}')

# Avaliar o modelo no conjunto de validação
evaluate_model(model, dataloaders, device)

Validation Accuracy: 0.7056


In [18]:
def evaluate_model_with_metrics(model, dataloaders, device, class_names):
    model.eval()  # Colocar o modelo em modo de avaliação
    running_corrects = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            running_corrects += torch.sum(preds == labels.data)

    accuracy = running_corrects.double() / len(dataloaders['val'].dataset)
    print(f'Validation Accuracy: {accuracy:.4f}')

    # Calcular e exibir a matriz de confusão
    cm = confusion_matrix(all_labels, all_preds)
    print("Matriz de Confusão:")
    print(cm)

    # Relatório de classificação
    print(classification_report(all_labels, all_preds, target_names=class_names))

# Função de avaliação com métricas
class_names = dataloaders['train'].dataset.classes  # Obtém os nomes das classes
evaluate_model_with_metrics(model, dataloaders, device, class_names)


Validation Accuracy: 0.7056
Matriz de Confusão:
[[109   0   0 ...   0   0   0]
 [  0  22   0 ...   0   0   0]
 [  0   1 199 ...   0   0   0]
 ...
 [  1   0   0 ...  28   0   0]
 [  0   0   0 ...   0  11   0]
 [  1   0   0 ...   0   0 114]]
                              precision    recall  f1-score   support

             Acacia_dealbata       0.39      0.84      0.54       129
         Acalypha_wilkesiana       0.61      0.73      0.67        30
       Aegopodium_podagraria       0.63      0.67      0.65       299
                 Alcea_rosea       0.61      0.87      0.71       536
          Alliaria_petiolata       0.84      0.77      0.80       793
       Alocasia_macrorrhizos       0.71      0.69      0.70        35
         Alocasia_sanderiana       1.00      0.89      0.94        45
         Althaea_officinalis       0.80      0.34      0.48        94
              Anemone_alpina       0.72      0.68      0.70       169
              Anemone_blanda       1.00      0.42      0.59

In [19]:
def evaluate_model_with_f1(model, dataloaders, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average='weighted')  # Ajuste a média conforme necessário
    print(f'F1 Score: {f1:.4f}')

In [20]:
def evaluate_model_with_precision_recall(model, dataloaders, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}')

In [21]:
# Avaliar o modelo no conjunto de validação com métricas
evaluate_model_with_f1(model, dataloaders, device)
evaluate_model_with_precision_recall(model, dataloaders, device)

F1 Score: 0.7057
Precision: 0.7447, Recall: 0.7056


In [22]:
def evaluate_model_with_metrics(model, dataloaders, device, class_names):
    model.eval()  # avaliação
    running_corrects = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            running_corrects += torch.sum(preds == labels.data)

    accuracy = running_corrects.double() / len(dataloaders['val'].dataset)
    print(f'Validation Accuracy: {accuracy:.4f}')

    # Relatório de classificação
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    print(report)

    # Salvar as métricas em um arquivo JSON
    with open('classification_report.json', 'w') as json_file:
        json.dump(report, json_file)

    # Converter o relatório e salvar em CSV
    df = pd.DataFrame(report).transpose()
    df.to_csv('classification_report.csv', index=True)

# função de avaliação com métricas
class_names = dataloaders['train'].dataset.classes  # Obtém os nomes das classes
evaluate_model_with_metrics(model, dataloaders, device, class_names)


Validation Accuracy: 0.7056
{'Acacia_dealbata': {'precision': 0.3920863309352518, 'recall': 0.8449612403100775, 'f1-score': 0.5356265356265356, 'support': 129.0}, 'Acalypha_wilkesiana': {'precision': 0.6111111111111112, 'recall': 0.7333333333333333, 'f1-score': 0.6666666666666666, 'support': 30.0}, 'Aegopodium_podagraria': {'precision': 0.6257861635220126, 'recall': 0.6655518394648829, 'f1-score': 0.6450567260940032, 'support': 299.0}, 'Alcea_rosea': {'precision': 0.6078431372549019, 'recall': 0.8675373134328358, 'f1-score': 0.7148347425057648, 'support': 536.0}, 'Alliaria_petiolata': {'precision': 0.8397260273972603, 'recall': 0.7730138713745272, 'f1-score': 0.8049901510177282, 'support': 793.0}, 'Alocasia_macrorrhizos': {'precision': 0.7058823529411765, 'recall': 0.6857142857142857, 'f1-score': 0.6956521739130435, 'support': 35.0}, 'Alocasia_sanderiana': {'precision': 1.0, 'recall': 0.8888888888888888, 'f1-score': 0.9411764705882353, 'support': 45.0}, 'Althaea_officinalis': {'precisi

In [23]:
torch.save(model.state_dict(), 'C:/Users/samue/codigos/IT_PLANTAS/pesos_it_1.pth')

In [24]:
torch.save(model, 'C:/Users/samue/codigos/IT_PLANTAS/modelo_it_1.pth')

In [25]:
from torchvision import models

# Recria a estrutura do modelo ResNet50 com a última camada ajustada para o seu número de classes
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Carrega os pesos salvos
model.load_state_dict(torch.load('C:/Users/samue/codigos/IT_PLANTAS/pesos_it_1.pth',weights_only=True))

# Coloca o modelo em modo de avaliação
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [26]:
from PIL import Image
import torch
import torchvision.transforms as transforms

# Função para carregar e transformar a imagem
def carregar_e_transformar_imagem(caminho_imagem):
    imagem = Image.open(caminho_imagem)

    # Verificar o modo da imagem e converter se necessário
    if imagem.mode != 'RGB':
        imagem = imagem.convert('RGB')

    # Transformar a imagem para o formato de entrada
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    input_tensor = transform(imagem).unsqueeze(0)  # Adiciona uma dimensão para o batch
    return input_tensor

# Caminho da imagem 
caminho_imagem = "C:/Users/samue/it_plantas/images_filtered_train/Aegopodium_podagraria/000c53532c67467eaefe462bee73b1c0cbbbb6b6.jpg"

# Carregar e transformar a imagem
input_tensor = carregar_e_transformar_imagem(caminho_imagem).to(device)

# Mover o modelo para o dispositivo
model = model.to(device)

# Verificar as classes e definir o número de classes
class_names = image_datasets['train'].classes  # Obter as classes do conjunto de treino
num_classes = len(class_names)
print(f"Classes encontradas: {class_names}")
print(f"Quantidade de classes: {num_classes}")

# Faz a previsão
with torch.no_grad():
    output = model(input_tensor)
    _, predicted = torch.max(output, 1)

# Obter a classe prevista
classe_prevista = class_names[predicted.item()]
print("Classe prevista:", classe_prevista)

Classes encontradas: ['Acacia_dealbata', 'Acalypha_wilkesiana', 'Aegopodium_podagraria', 'Alcea_rosea', 'Alliaria_petiolata', 'Alocasia_macrorrhizos', 'Alocasia_sanderiana', 'Althaea_officinalis', 'Anemone_alpina', 'Anemone_blanda', 'Anemone_coronaria', 'Anemone_hepatica', 'Anemone_hortensis', 'Anemone_hupehensis', 'Anemone_narcissiflora', 'Anemone_nemorosa', 'Anemone_pulsatilla', 'Anemone_ranunculoides', 'Anemone_vernalis', 'Angelica_sylvestris', 'Anthericum_liliago', 'Anthurium_andraeanum', 'Aphelandra_squarrosa', 'Asystasia_gangetica', 'Barbarea_vulgaris', 'Butomus_umbellatus', 'Calendula_officinalis', 'Calycanthus_floridus', 'Carthamus_lanatus', 'Carthamus_tinctorius', 'Cenchrus_setaceus', 'Centranthus_ruber', 'Chaerophyllum_temulum', 'Cirsium_acaulon', 'Cirsium_arvense', 'Cirsium_eriophorum', 'Cirsium_oleraceum', 'Cirsium_palustre', 'Cirsium_spinosissimum', 'Cirsium_vulgare', 'Couroupita_guianensis', 'Cucurbita_pepo', 'Cymbalaria_muralis', 'Daphne_cneorum', 'Daphne_gnidium', 'Daph

In [3]:
import torchvision.transforms as transforms
from torchvision import models
from torchvision.models import ResNet50_Weights
import gradio as gr

# Obter os nomes das classes diretamente do diretório de treino
class_names = sorted(os.listdir(train_dir))  # Obtém e ordena as pastas por nome científico
num_classes = len(class_names)

# Carrega o modelo com os pesos salvos e define as classes
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, num_classes)
model.load_state_dict(torch.load('C:/Users/samue/codigos/IT_PLANTAS/pesos_it_1.pth',weights_only=True, map_location=torch.device('cpu')))
model.eval()

# Define a transformação da imagem
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Função para classificar a imagem
def classify_image(image):
    image = preprocess(image)
    image = image.unsqueeze(0)  # Adiciona uma dimensão para o batch
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    # Obter o nome científico da classe prevista usando o índice
    nome_planta = class_names[predicted.item()]  # Nome da classe previsto

    return nome_planta  # Retorna o nome científico da planta

# Cria a interface Gradio
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Textbox(),
    title="Classificação de Imagens",
    description="Carregue uma imagem e veja a classe prevista."
)

# Inicia a interface
iface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
